References :
1> Campuswire : https://campuswire.com/c/G06C55090/feed/432 
2> Campuswire : https://campuswire.com/c/G06C55090/feed/458
3> Lecture: https://liangfgithub.github.io/Notes/lec_W7.2_HMM.pdf

Assignment : https://liangfgithub.github.io/Coding/Coding4.html

## Hints

https://campuswire.com/c/G06C55090/feed/612
Hi All,

I'm struggling in debugging coding4 part2 Baum-Welch algorithm. I first calculated alpha (dim [T-1, mz]), beta (dim [T, mz]), gamma (dim [T-1, mz, mz]), and margiGamma (dim [T, mz]), then I updated the A (mz-by-mz) and B (mz-by-mx) according to the hints in #458. Following is my resulted alpha, beta, A, and B for the first iteration. The results are inaccurate and I have no idea which part went wrong. Has anyone met the same issue? Any help or hints will be appreciated!

I finally found the bug that when I read the observations I used "myData <-read.csv("https://liangfgithub.github.io/Data/coding4_part2_data.txt", header = True)" which omitted the first observation. After change header to FALSE, I can get the correct A and B now.
It is also helpful to run the R baumWelch "myfit2 = baumWelch(tmphmm, myData[,1], 2)" to debug with desired iteration numbers.

## Implementation

In [2]:
import numpy as np

In [111]:
# Load the data sequence from the file
data = np.genfromtxt("coding4_part2_data.txt", dtype=int)
w = np.array([0.5, 0.5])
A = np.array([[0.5, 0.5], [0.5, 0.5]])
B = np.array([[1/9, 3/9, 5/9], [1/6, 2/6, 3/6]])

In [112]:
def compute_forward_probability(t, i,w, alpha, A, B, x):
    #print(f"t={t} , i : {i} , alpha: {alpha.shape}")

    if t == 1 and alpha[t - 1][i] == 0:
        #print(f"t=1 , i : {i} , alpha_t_i: {alpha[t - 1][i]} , w_{i} :{w[i]} ,B[{i}][x{t-1}-1] : {B[i][x[t - 1]-1]} ")
        # Base case: When t = 1
        alpha[t - 1][i] = w[i] * B[i][x[t - 1]-1]
        #print(f"alpha_t_i: {alpha} ")
        return alpha[t - 1][i]
    elif alpha[t - 1][i] != 0:
        # If the value is already calculated, return it
        return alpha[t - 1][i]
    else:
        # Recursive case: When t > 1
        summation = 0
        for j in range(w.shape[0]):  # Make sure A has the correct number of rows
            alpha_temp = compute_forward_probability(t-1, j,w, alpha, A, B, x)
            #print(f"t={t} , i : {i} ,j:{j}, alpha_temp: {alpha_temp} ,A[{j}][i]: {A[j][i]},B[{i}][x{t-1}-1] : {B[i][x[t - 1]-1]} ")

            summation +=  alpha_temp * A[j][i] * B[i][x[t - 1]-1]  # Swap i and j here
        alpha[t - 1][i] = summation  # Store the calculated value in the alpha matrix
        #print(f"t={t} , i : {i},alpha_t+: {alpha} ")
        return summation


In [162]:
def compute_backward_probability(t, i, beta, A, B, x):
    print(f"t={t} , i : {i} , beta: {beta.shape}, len(x):{len(x)}")
    if t == (len(x)):
        # Base case: When t is the last time step
        beta[t - 1] = 1 #[i]
        print(f"beta[t - 1]:{beta[t - 1]}")
        return 1
    elif beta[t - 1][i] != 0:
        # If the value is already calculated, return it
        #print(f"beta[t - 1]:{beta[t - 1][i]}")
        return beta[t - 1][i]
    else:
        # Recursive case: When t < T
        summation = 0
        for j in range(A.shape[0]):
            summation += A[i][j] * B[j][x[t]-1] * compute_backward_probability(t + 1, j, beta, A, B, x)
        beta[t - 1][i] = summation  # Store the calculated value in the beta matrix
        #print(f"summation:{summation}")
        return summation


In [163]:
def backward_probability_recursive(t, i, beta, A, B, x):
    if t == len(x):
        # Base case: When t is the last time step
        beta[t - 1] = 1
        return 1
    elif beta[t - 1][i] != 0:
        # If the value is already calculated, return it
        return beta[t - 1][i]
    else:
        # Recursive case: When t < T
        summation = 0
        for j in range(A.shape[0]):
            summation += A[i][j] * B[j][x[t]-1] * backward_probability_recursive(t + 1, j, beta, A, B, x)
        beta[t - 1][i] = summation  # Store the calculated value in the beta matrix
        return summation


In [164]:
def compute_gamma_onestep(data, w,A,B):
    T = len(data)
    wz = w.shape[0]
    print(f"T:{T}, wz:{wz}")
    
    alpha_one_step = np.zeros((T, wz))
    #print(f"alpha_one_step: {alpha_one_step.shape} ")
    compute_forward_probability(T, wz -1,w, alpha_one_step,A,B, data )
    
    #print(f"alpha_one_step: {alpha_one_step.shape} , alpha_one_step\n-----\n{alpha_one_step[1:10,]} ")
    
    # Initialize the beta array
    beta_one_step = np.zeros((len(data), A.shape[0]))
    
    print(f"beta_one_step: {beta_one_step.shape} ")
    print(f"data: {data}")
    beta_t_i = compute_backward_probability(T,wz-1 ,beta_one_step,A,B, data)
    #print("β{}({}) = {}".format(T, wz, beta_t_i))
    print(f"beta_one_step: {beta_one_step.shape} , beta_one_step\n-----\n{beta_one_step[1:10,]} ")

In [165]:
def BW_onestep(data, w,A,B):
    
    # STEP 1: E step
    myGamma = compute_gamma_onestep(data, w,A,B)
    # Step 2: M Step
    
    # return updated parameters A, B
    #return A, B

In [166]:
BW_onestep(data, w,A,B)

T:200, wz:2
beta_one_step: (200, 2) 
data: [2 3 3 3 3 3 3 1 3 3 3 3 3 1 1 3 3 3 2 3 3 3 1 3 1 1 1 2 2 3 3 3 3 3 1 3 3
 2 2 3 3 3 3 1 1 3 3 3 2 3 3 1 3 1 3 3 3 2 1 1 1 1 3 3 3 3 3 3 3 3 1 1 1 3
 3 1 2 1 3 1 1 3 1 1 2 1 3 1 1 2 1 3 3 3 3 1 3 3 3 3 3 3 2 3 3 2 3 2 3 1 3
 3 3 3 1 2 2 3 3 3 3 1 2 2 2 1 3 1 3 1 1 3 1 2 3 3 3 2 2 3 1 3 1 2 1 2 1 3
 1 3 1 3 3 3 1 1 3 2 3 1 3 3 2 3 3 1 1 1 2 3 1 1 2 1 3 3 3 1 2 1 3 2 3 3 3
 3 3 2 1 2 2 3 1 2 3 2 3 2 2 2]
t=200 , i : 1 , beta: (200, 2), len(x):200
beta[t - 1]:[1. 1.]
beta_one_step: (200, 2) , beta_one_step
-----
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 


In [167]:
w1 = np.array([0.5, 0.5])
A1 = np.array([[0.5, 0.5], [0.5, 0.5]])
B1 = np.array([[1/9, 3/9, 5/9], [1/6, 2/6, 3/6]])
x1 = np.array([2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 2, 3, 3, 3, 1, 3, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 1, 3,
 2, 2, 3, 3, 3, 3, 1, 1, 3, 3, 3, 2, 3, 3, 1, 3, 1, 3, 3, 3, 2, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3,
 3, 1, 2, 1, 3, 1, 1, 3, 1, 1, 2, 1, 3, 1, 1, 2, 1, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2, 3, 1, 3,
 3, 3, 3, 1, 2, 2, 3, 3, 3, 3, 1, 2, 2, 2, 1, 3, 1, 3, 1, 1, 3, 1, 2, 3, 3, 3, 2, 2, 3, 1, 3, 1, 2, 1, 2, 1, 3,
 1, 3, 1, 3, 3, 3, 1, 1, 3, 2, 3, 1, 3, 3, 2, 3, 3, 1, 1, 1, 2, 3, 1, 1, 2, 1, 3, 3, 3, 1, 2, 1, 3, 2, 3, 3, 3,
 3, 3, 2, 1, 2, 2, 3, 1, 2, 3, 2, 3, 2, 2])

# Define the time step 't' and the hidden state 'i' for which you want to calculate βt(i)
t = 1  # For example, calculate βt(1)

# Initialize the beta array
beta1 = np.zeros((len(x1), A1.shape[0]))

# Calculate βt(i) using the recursive function
beta_t_i = backward_probability_recursive(len(x1), len(w), beta1, A1, B1, x1)

print("β{}({}) = {}".format(len(x1), len(w), beta_t_i))


β198(2) = 1


In [168]:
beta1

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.

In [169]:
beta1 = np.zeros((len(data), A.shape[0]))
print(w.shape[0])
for t in range(len(data)):
    for i in range(w.shape[0]):
        compute_backward_probability(t, i, beta1, A, B, data)
        
beta1.shape

2
t=0 , i : 0 , beta: (200, 2), len(x):200
t=1 , i : 0 , beta: (200, 2), len(x):200
t=2 , i : 0 , beta: (200, 2), len(x):200
t=3 , i : 0 , beta: (200, 2), len(x):200
t=4 , i : 0 , beta: (200, 2), len(x):200
t=5 , i : 0 , beta: (200, 2), len(x):200
t=6 , i : 0 , beta: (200, 2), len(x):200
t=7 , i : 0 , beta: (200, 2), len(x):200
t=8 , i : 0 , beta: (200, 2), len(x):200
t=9 , i : 0 , beta: (200, 2), len(x):200
t=10 , i : 0 , beta: (200, 2), len(x):200
t=11 , i : 0 , beta: (200, 2), len(x):200
t=12 , i : 0 , beta: (200, 2), len(x):200
t=13 , i : 0 , beta: (200, 2), len(x):200
t=14 , i : 0 , beta: (200, 2), len(x):200
t=15 , i : 0 , beta: (200, 2), len(x):200
t=16 , i : 0 , beta: (200, 2), len(x):200
t=17 , i : 0 , beta: (200, 2), len(x):200
t=18 , i : 0 , beta: (200, 2), len(x):200
t=19 , i : 0 , beta: (200, 2), len(x):200
t=20 , i : 0 , beta: (200, 2), len(x):200
t=21 , i : 0 , beta: (200, 2), len(x):200
t=22 , i : 0 , beta: (200, 2), len(x):200
t=23 , i : 0 , beta: (200, 2), len(x):200


(200, 2)

In [170]:
beta1.shape

(200, 2)

In [171]:
beta1[199]

array([7.22446254e-97, 1.00000000e+00])